# <span style="color:brown"> **Hands-on Deep Learning – Interpretability of Image Classification**

_This hands-on was created by Thomas Grenier, CREATIS_

thomas.grenier@creatis.insa-lyon.fr,
michael.sdika@creatis.insa-lyon.fr

# <span style="color:brown"> **Introduction on data**

This hands-on is computer scientist-oriented more than application-oriented. 

It is the second part of the classification hands-on: we will use the same data and study GradCam.
The first cells are the same as the first notebook.
Changes occur in section "E"

> Let's go!

# <span style="color:brown"> **A - Setup**

### A1 - First, import a few common modules, ensure MatplotLib plots figures inline, and functions for converting data and displaying reports:

In [ ]:
# Common imports
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np

!pip install tqdm
from tqdm import tqdm

!pip install scikit-learn
from sklearn import datasets, metrics

### A2- We import TensorFlow 2

In [ ]:
import tensorflow as tf
print(tf.__version__)

> _**If no error occurs, ie the version of TensorFlow is printed, your working environment is ok and you can go to the next part,
> else ... call an assistant for help!**_

# <span style="color:brown"> **B - Read the data**

<span style="color:red">
    

We now use a function to do most of the work on data.

In [ ]:
from tools import ReadHandsOnData
from tools import DisplayRGBImages

Some important values and initializations:

In [ ]:
desired_height = 64
desired_width  = 64

class_names = ['PD-A', 'PD-C', 'PD-S', 'T1-A', 'T1-C', 'T1-S', 'T2-A', 'T2-C', 'T2-S']

# Definition of constants
input_shape = (desired_height,desired_width,1)
input_shape_rgb = (desired_height,desired_width,3)
num_classes = 9

### <span style="color:brown"> **B1 - Load data**

In [ ]:
dataset="dlss_classification_A"

X_train_rgb, y_train, X_val_rgb, y_val = ReadHandsOnData('train_val', randomize_image=True, validation_ratio=0.2, desired_height=desired_height, desired_width=desired_width, class_names=class_names, file_link = dataset)
X_test_rgb, y_test= ReadHandsOnData('test', randomize_image=False, validation_ratio=0, desired_height=desired_height, desired_width=desired_width, class_names=class_names, file_link = dataset)
DisplayRGBImages(X_val_rgb, y_val, nb_x = 5, nb_y = 5, class_names=class_names)

### <span style="color:brown"> **B2 - Convert the data shape and type**
The _png_ files are RGB, we convert them to gray images for each set (train, validation and test) in order to be compatible with all tested approaches.

We also divide intensity values by 255 to bound range in [0;1]


In [ ]:
# converting the three data set : rgb to gray scale images, then normalization

X_train = X_train_rgb[:,:,:,1].astype(np.float32)/255
X_train = np.expand_dims(X_train,-1)

X_val = X_val_rgb[:,:,:,1].astype(np.float32)/255
X_val = np.expand_dims(X_val,-1)

X_test = X_test_rgb[:,:,:,1].astype(np.float32)/255
X_test = np.expand_dims(X_test,-1)

print(" Shape : ", X_train.shape, X_val.shape, X_test.shape)
print(" Type  : %s  %s %s"%(X_train.dtype, X_val.dtype, X_test.dtype))
print(" Max   : %d %d %d"%(X_train.max(), X_val.max(), X_test.max()) )
print(" Min   : %d %d %d"%(X_train.min(),X_val.min(), X_test.min()) )

In [ ]:
y_train_cat = tf.keras.utils.to_categorical(y_train)
y_val_cat = tf.keras.utils.to_categorical(y_val)
y_test_cat = tf.keras.utils.to_categorical(y_test)

print(" Original (sparse) shape  : ", y_train.shape, y_val.shape, y_test.shape)
print(" Categorical shape  : ", y_train_cat.shape, y_val_cat.shape, y_test_cat.shape)
print(" Example : ", y_train[0,], " --> " ,y_train_cat[0,])

# Now changing name :
y_train = y_train_cat
y_val = y_val_cat
y_test = y_test_cat

> _**All done, go to the classification step!**_

# <span style="color:brown"> **C - Classification**

### Function for displaying reports

In [ ]:
# Usage example : ClassificationReport(model, X_test, y_test_cat , class_names)
def ClassificationReport(model, X_test, y_test_cat, labels_name):  
    # predict 
    print("Prediction on test images")
    predicted = model.predict(X_test)
    y_pred = np.argmax(predicted, axis=-1)
    y_test = np.argmax(y_test_cat, axis=-1)
    
    print("Classification report :\n%s\n"
         % (metrics.classification_report(y_test, y_pred)) )
    cm = metrics.confusion_matrix( y_test, y_pred )
    disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm,  display_labels=labels_name)
    disp.plot()
    disp.figure_.suptitle("Confusion Matrix")
    plt.show()

## <span style="color:brown"> **D - CNN Classification**


### <span style="color:brown"> **D2 - Convolutional Neural Network (CNN)**

In order to try to improve the results, we are now looking at convolutional neural networks, of which an example architecture is provided below.

<img src="./figures/architecture_cnn_en.png" alt="Architecture CNN" style="width: 90%"/>


In [ ]:
# Definition of variables that you will have to play with during the session to study their influence

# Variables related to the FCN architecture
nb_mlp_neurons = 16    # Number of neurons for one layer
nb_cnn_neurons = 16    # number of feature maps

# Variables related to the optimization process
batch_size  = 16       # Number of images for each batch during the training process 
nb_epochs   = 15       # number of epochs used during the training process
learning_rate = 0.001  # learning rate used during the training process

Run the cell below to create a CNN (Convolutional Neural Network) model using a sequential declaration allowed by the tensorflow library

In [ ]:
# This part of the code allows you to free up memory space 
if 'model' in locals(): 
    print("deleting model")
    del model
    
# Define a CNN model from sequential declaration    
model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=input_shape),
  tf.keras.layers.Conv2D(nb_cnn_neurons//2, 3, activation='relu', padding='same'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(nb_cnn_neurons, 3, activation='relu', padding='same'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(nb_cnn_neurons*2, 3, activation='relu', padding='same'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(nb_mlp_neurons, activation='relu'),  #128
  tf.keras.layers.Dense(num_classes, activation = "softmax")
])

# Display the network structure on the console
model.summary()

Run the cell below to train the CNN model

In [ ]:
# Compile (define the optimization scheme) the model to be ready for training
from tensorflow.keras import optimizers
from tensorflow import losses

model.compile(
  optimizer=optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
  loss=tf.losses.CategoricalCrossentropy(),
  metrics=['accuracy'])

# Train the model
model.fit(
  X_train, y_train,
  validation_data=(X_val, y_val),
  batch_size=batch_size,  
  epochs=nb_epochs
)

Run the cell below to evaluate the performance of the model on the test dataset.

In [ ]:
# Compute the overall accuracy of the model on the test dataset 
test_loss, test_accuray = model.evaluate(X_test, y_test, verbose=1)
print('Test accuracy:', test_accuray)

# Compute and display standart classication metrics, i.e. precision, recall, f1-score, support and confusion matrix 
ClassificationReport(model, X_test, y_test, class_names)

## <span style="color:brown"> **E - Why it works so good?**

Can we analyze how the network analyze the images? 

In some way the answer is yes, partially. This is the aim of __explainability__ and __interpretability__.
    
 
The idea is to observe which part of the image influence the final classification decision.
For a given class and a given image, which pixels contributes to activate the last layer?
Many methods have been proposed (GradCAM, intergrated CAM , see the work of Linardatos 2021 https://www.mdpi.com/1099-4300/23/1/18/pdf ).
    
Here we propose to use the GradCam approach to explore which pixels contributes to the final class activation.
(code largely inspired from https://keras.io/examples/vision/grad_cam/)
    

In [ ]:
def make_gradcam_heatmap(img_array, last_conv_layer_name, model, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.outputs[0]]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    
    # This is the gradient of the output neuron (top predicted or chosen)
    # about the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" concerning the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


Then a function to display the heatmap

In [ ]:
import matplotlib.cm as cm

def display_gradcam(img_array, heatmap, alpha=0.4):
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)
    img_array = np.uint8(255 * img_array)
    
    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img_array.shape[2], img_array.shape[1]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img_array[0]
    return np.float32(superimposed_img/255)

### <span style="color:brown"> **E2 - Exploring activation maps**
    
Now explore the activation map of a given image and the last convolutional layer output

In [ ]:
last_conv_layer_name = 'conv2d_2'    # <----- selected layer (simple network)
img_array = X_test[1441:1442]               # <----- selected image
print(img_array.shape)

# Remove the last layer's softmax
model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)

print("Predicted outputs:", preds)

In [ ]:
print("arg max of Predicted:", np.argmax(preds))
index_pred = np.argmax(preds)      # <----- select the index you want to analyze how it has been activated

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, last_conv_layer_name, model, index_pred )

print(heatmap.shape)

In [ ]:
# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
active_img = display_gradcam(img_array, heatmap)
plt.imshow(active_img, aspect="equal", interpolation="nearest")

### <span style="color:red"> **Questions**
- When index_pred is the arg max of the predictions, where are the pixels that activate this class? but ... What did we expect?
- Conclusions? Why the image classification algorithms work so well?
- Can you propose modifications on input images to remove this bias?

### <span style="color:red"> **Questions**
In cell 15, change the dataset from dataset="dlss_classification_A" to dataset="dlss_classification_D" and restart the kernel then rerun the notebook.

What are your main observations about:
- the images,
- the number of images, 
- the accuracy and f1-score,
- the explainability ?


### <span style="color:brown"> **Now :** 
### <span style="color:brown">        **1- Shutdown the kernel**
(menu --> Kernel --> "Shut down kernel")

### <span style="color:brown">        **2- Open the next notebook!**
04_... To discover how we can improve classification scores on this modified dataset with small nmodels!